In [284]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np
from google.colab import drive


In [285]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [286]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KaggleComp/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KaggleComp/test.csv')

In [287]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [288]:
# Setting up the data, cleaning/deciding important variables
y_train = df_train['Survived']

x_train = df_train.set_index('PassengerId').drop(['Survived', 'Name', 'Cabin', 'Ticket'], axis=1)
x_test = df_test.copy().set_index('PassengerId').drop(['Name', 'Cabin', 'Ticket'], axis=1)
test_results = x_test

x_train = pd.get_dummies(x_train).fillna(0)
x_test = pd.get_dummies(x_test).fillna(0)



In [289]:
one_hot = pd.get_dummies(x_train['Pclass'])
# Drop column B as it is now encoded
#df_train = df_train.drop('Pclass',axis = 1)
# Join the encoded df
x_train = x_train.join(one_hot)
x_train  

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3
PassengerId,,,,,,,,,,,,,
1,3,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
2,1,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
3,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
4,1,35.0,1,0,53.1000,1,0,0,0,1,1,0,0
5,3,35.0,0,0,8.0500,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,27.0,0,0,13.0000,0,1,0,0,1,0,1,0
888,1,19.0,0,0,30.0000,1,0,0,0,1,1,0,0
889,3,0.0,1,2,23.4500,1,0,0,0,1,0,0,1


In [290]:
one_hot2 = pd.get_dummies(x_test['Pclass'])
# Drop column B as it is now encoded
#df_train = df_train.drop('Pclass',axis = 1)
# Join the encoded df
x_test = x_test.join(one_hot2)
x_test 

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3
PassengerId,,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0,1,0,1,0,0,0,1
893,3,47.0,1,0,7.0000,1,0,0,0,1,0,0,1
894,2,62.0,0,0,9.6875,0,1,0,1,0,0,1,0
895,3,27.0,0,0,8.6625,0,1,0,0,1,0,0,1
896,3,22.0,1,1,12.2875,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,0.0,0,0,8.0500,0,1,0,0,1,0,0,1
1306,1,39.0,0,0,108.9000,1,0,1,0,0,1,0,0
1307,3,38.5,0,0,7.2500,0,1,0,0,1,0,0,1


In [291]:
from sklearn.preprocessing import StandardScaler
# create scaler
scaler = StandardScaler()
# transformed the feature
standardized_age = scaler.fit_transform(x_train[['Age']])
standardized_fare = scaler.fit_transform(x_train[['Fare']])
x_train['Standard Age'] = standardized_age
x_train['Standard Fare'] = standardized_fare
x_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3,Standard Age,Standard Fare
PassengerId,,,,,,,,,,,,,,,
1,3,22.0,1,0,7.2500,0,1,0,0,1,0,0,1,-0.102313,-0.502445
2,1,38.0,1,0,71.2833,1,0,1,0,0,1,0,0,0.807492,0.786845
3,3,26.0,0,0,7.9250,1,0,0,0,1,0,0,1,0.125138,-0.488854
4,1,35.0,1,0,53.1000,1,0,0,0,1,1,0,0,0.636903,0.420730
5,3,35.0,0,0,8.0500,0,1,0,0,1,0,0,1,0.636903,-0.486337


In [292]:
from sklearn.preprocessing import StandardScaler
# create scaler
scaler = StandardScaler()
# transformed the feature
standardized_age2 = scaler.fit_transform(x_test[['Age']])
standardized_fare2 = scaler.fit_transform(x_test[['Fare']])
x_test['Standard Age'] = standardized_age2
x_test['Standard Fare'] = standardized_fare2
x_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3,Standard Age,Standard Fare
PassengerId,,,,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0,1,0,1,0,0,0,1,0.594801,-0.496637
893,3,47.0,1,0,7.0000,1,0,0,0,1,0,0,1,1.305896,-0.511497
894,2,62.0,0,0,9.6875,0,1,0,1,0,0,1,0,2.159209,-0.463335
895,3,27.0,0,0,8.6625,0,1,0,0,1,0,0,1,0.168145,-0.481704
896,3,22.0,1,1,12.2875,1,0,0,0,1,0,0,1,-0.116293,-0.416740


In [293]:
x_test = x_test.drop('Pclass',axis = 1)
x_test = x_test.drop('Age', axis=1)
x_test = x_test.drop('Fare', axis=1)
x_train = x_train.drop('Pclass',axis = 1)
x_train = x_train.drop('Age', axis=1)
x_train = x_train.drop('Fare', axis=1)

In [294]:
# Creating Neural Network model
clf = MLPClassifier(hidden_layer_sizes=(10), solver='lbfgs', activation='logistic', random_state=42)
clf.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)
print('Accuracy on training set:', accuracy_score(y_train, y_pred_train))

y_pred_test = clf.predict(x_test)

test_results['Survived'] = y_pred_test
test_results = test_results[['Survived']].reset_index()
test_results

Accuracy on training set: 0.8754208754208754


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [295]:
# Creating Random Forests model
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(max_depth=5)
clf_rf.fit(x_train, y_train)
clf_rf_pred_train = clf_rf.predict(x_train)
print('Accuracy from random forest:', accuracy_score(y_train, clf_rf_pred_train))

clf_rf_pred_test = clf_rf.predict(x_test)
test_results['Survived'] = clf_rf_pred_test
test_results

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Accuracy from random forest: 0.8417508417508418


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [296]:
# Exporting the test predictions
from google.colab import files
test_results.to_csv('out1.csv', index=False)
files.download('out1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [297]:
# Creating Logistic Regression model
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(max_iter = 1000).fit(x_train, y_train)
clf_lr_pred_train = clf_lr.predict(x_train)

accuracy_score(y_train, clf_lr_pred_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.7968574635241302

In [298]:
# # Exporting the test predictions
# from google.colab import files
# test_results.to_csv('out.csv', index=False)
# files.download('out.csv')

Exploratory Data Analysis (EDA)

Checklist:
1. Remove duplicate entries - doesn't apply bc they're asking us to submit all 418 entries
2. Investigate null values: what do they mean, which columns are they in, address/remove/replace them

*   age: 177
*   cabin: 687
*   embarked: 2


In [299]:
df_train.head(30)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [300]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [301]:
age_df = df_train.groupby('Age').count().reset_index()
sns.set(rc={"figure.figsize":(8, 6)})
sns.lineplot(data=age_df, x=age_df['Age'], y=age_df['Survived'])


NameError: ignored

In [302]:
fare_df = df_train.groupby('Fare').count().reset_index()
sns.set(rc={"figure.figsize":(8, 6)})
sns.lineplot(data=fare_df, x=fare_df['Fare'], y=fare_df['Survived'], color='red')


NameError: ignored

In [303]:
pclass_df = df_train.groupby('Pclass').count().reset_index()
#sns.set(rc={"figure.figsize":(8, 6)})
sns.lineplot(data=pclass_df, x=pclass_df['Pclass'], y=pclass_df['Survived'], color='green')

NameError: ignored

In [304]:
x_train

,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3,Standard Age,Standard Fare
PassengerId,,,,,,,,,,,,
1,1,0,0,1,0,0,1,0,0,1,-0.102313,-0.502445
2,1,0,1,0,1,0,0,1,0,0,0.807492,0.786845
3,0,0,1,0,0,0,1,0,0,1,0.125138,-0.488854
4,1,0,1,0,0,0,1,1,0,0,0.636903,0.420730
5,0,0,0,1,0,0,1,0,0,1,0.636903,-0.486337
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,0,0,1,0,0,1,0,1,0,0.182001,-0.386671
888,0,0,1,0,0,0,1,1,0,0,-0.272901,-0.044381
889,1,2,1,0,0,0,1,0,0,1,-1.353294,-0.176263


In [305]:
x_test

,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3,Standard Age,Standard Fare
PassengerId,,,,,,,,,,,,
892,0,0,0,1,0,1,0,0,0,1,0.594801,-0.496637
893,1,0,1,0,0,0,1,0,0,1,1.305896,-0.511497
894,0,0,0,1,0,1,0,0,1,0,2.159209,-0.463335
895,0,0,0,1,0,0,1,0,0,1,0.168145,-0.481704
896,1,1,1,0,0,0,1,0,0,1,-0.116293,-0.416740
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0,0,1,0,0,1,0,0,1,-1.367819,-0.492680
1306,0,0,1,0,1,0,0,1,0,0,0.850795,1.314641
1307,0,0,0,1,0,0,1,0,0,1,0.822352,-0.507017


In [316]:
test_df = x_test.reset_index()[['PassengerId']]
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
              ('mlp', MLPClassifier(hidden_layer_sizes=(10), solver='lbfgs', activation='logistic',max_iter = 10000,random_state=42))]
clf_stack = StackingClassifier(estimators= estimators, final_estimator=LogisticRegression())
clf_stack.fit(x_train, y_train)
clf_stack_pred_train = clf_stack.predict(x_train)
print('Accuracy from Stacking:', accuracy_score(y_train, clf_stack_pred_train))
y_pred_stack_test = clf_stack.predict(x_test)

test_df['Survived'] = y_pred_stack_test
test_df

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

Accuracy from Stacking: 0.867564534231201


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [317]:
# Exporting the test predictions
from google.colab import files
test_df.to_csv('out1.csv', index=False)
files.download('out1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [308]:
x_train

,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,1,2,3,Standard Age,Standard Fare
PassengerId,,,,,,,,,,,,
1,1,0,0,1,0,0,1,0,0,1,-0.102313,-0.502445
2,1,0,1,0,1,0,0,1,0,0,0.807492,0.786845
3,0,0,1,0,0,0,1,0,0,1,0.125138,-0.488854
4,1,0,1,0,0,0,1,1,0,0,0.636903,0.420730
5,0,0,0,1,0,0,1,0,0,1,0.636903,-0.486337
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,0,0,1,0,0,1,0,1,0,0.182001,-0.386671
888,0,0,1,0,0,0,1,1,0,0,-0.272901,-0.044381
889,1,2,1,0,0,0,1,0,0,1,-1.353294,-0.176263


In [314]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(RandomForestClassifier(max_depth=5),
                            max_samples=0.5, max_features=0.5)

bagging.fit(x_train, y_train)
bagging_train_pred = bagging.predict(x_train)

print('Accuracy from bagging:', accuracy_score(y_train, bagging_train_pred))

bagging_test_pred = bagging.predict(x_test)

test_results['Survived'] = bagging_test_pred
test_results

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Accuracy from bagging: 0.8215488215488216


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [315]:
# Exporting the test predictions
from google.colab import files
test_results.to_csv('out1.csv', index=False)
files.download('out1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>